In [74]:
#import libraries
import pandas as pd
import numpy as np
import os

In [75]:
path= r'/Users/eleni_icon/09-2024 Instacart Basket Analysis'

In [76]:
# Define your path to the pickle file
path = r'/Users/eleni_icon/09-2024 Instacart Basket Analysis'

# Load the pickle file
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

# 1a. Create price_label column

In [77]:
df = df_ords_prods_merge[:1000000] 

In [78]:
df.shape

(1000000, 16)

In [79]:
# Define a function for price labels
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [105]:
df['price_range'] = df.apply(price_label, axis=1)


/var/folders/fs/0m_d2lxj6mdbrcp0xzk4cnjh0000gn/T/ipykernel_35891/3677927974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [106]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    673293
Low-range product    314432
High range            12275
Name: count, dtype: int64

In [107]:
df['prices'].max()


25.0

In [83]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'
df['price_range_loc'].value_counts(dropna=False)

/var/folders/fs/0m_d2lxj6mdbrcp0xzk4cnjh0000gn/T/ipykernel_35891/1202714276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


price_range_loc
Mid-range product     673293
Low-range product     314432
High-range product     12275
Name: count, dtype: int64

In [84]:
# Use loc() method on entire ords_prods_merge dataframe to apply price range function.

df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [85]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [86]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [87]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      412551
Name: count, dtype: int64

# 1b. Create busiest_day column

In [88]:
# Change "order_dow" column name to "order_day_of_week."

df_ords_prods_merge.rename(columns = {'order_dow' : 'orders_day_of_week'}, inplace = True)

In [89]:
# Find frequency of orders_day_of_week.

df_ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6203329
1    5659298
6    4495887
2    4213105
5    4205076
3    3839865
4    3783172
Name: count, dtype: int64

In [90]:
# Use for-loop to determine whether orders are on "busiest day" (0 = Saturday), "least busy" (4 = Wednesday), or "regularly busy" (other days of the week).

result = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [91]:
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy

In [92]:
# Create new column in ords_prods_merge to view results in context.

df_ords_prods_merge['busiest_day'] = result

In [93]:
df_ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22413231
Busiest day        6203329
Least busy         3783172
Name: count, dtype: int64

# 2. Update busiest day column to busiest days to identify order day of the week as Busiest days, Slowest days, or Regularly busy.

In [94]:
# Create column “busiest_days" from "orders_day_of_week" to identify two busiest days (0, 1), slowest days (4, 3), or regularly busy (all other days).

result_2 = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0 or value == 1:
    result_2.append("Busiest days")
  elif value == 4 or value == 3:
    result_2.append("Slowest days")
  else:
    result_2.append("Regularly busy")

In [95]:
result_2

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest day

# 3. Check values of new busiest days column for accuracy with observations in markdown format.

In [96]:
 #Create new 'busiest_days' column in ords_prods_merge to view results in context.

df_ords_prods_merge['busiest_days'] = result_2

In [97]:
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    12914068
Busiest days      11862627
Slowest days       7623037
Name: count, dtype: int64

In [98]:
# Check output of ords_prods_merge with new "busiest days" column.

df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,match,price_range_loc,busiest_day,busiest_days
0,2539329,1,1,2,8,NaN,True,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy
1,2539329,1,1,2,8,NaN,True,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Regularly busy
2,2539329,1,1,2,8,NaN,True,12427,3,0,both,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Regularly busy
3,2539329,1,1,2,8,NaN,True,26088,4,0,both,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Regularly busy
4,2539329,1,1,2,8,NaN,True,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Regularly busy


# 4. Create new column busiest_period_of_day to identify time periods Most orders, Average orders, and Fewest orders.

In [99]:
# Check value counts in "order_hour_of_day" column.

df_ords_prods_merge['order_hour_of_day'].value_counts()

order_hour_of_day
10    2761333
11    2735694
14    2688728
15    2661718
13    2660570
12    2618104
16    2534744
9     2453842
17    2087273
8     1717863
18    1636226
19    1258076
20     976000
7      890923
21     795528
22     634159
23     402272
6      290450
0      218742
1      115683
5       87944
2       69360
4       53232
3       51268
Name: count, dtype: int64

In [100]:
# Create for-loop if statement labeling periods of time as “Most orders,” “Average orders,” and “Fewest orders.”

result_3 = []

for value in df_ords_prods_merge["order_hour_of_day"]:
  if value in [10, 11, 14, 15, 13, 12, 16, 9]:
    result_3.append("Most orders")
  elif value in [23, 6, 0, 1, 5, 2, 4, 3]:
    result_3.append("Fewest orders")
  else:
    result_3.append("Average orders")

In [101]:
result_3

['Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Avera

In [102]:
# Create new column "busiest_period_of_day" in ords_prods_merge.

df_ords_prods_merge['busiest_period_of_day'] = result_3

# 5. Print frequency of busiest_period_of_day column.

In [103]:
# Print value counts in "busiest_period_of_day" column.

df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21114733
Average orders     9996048
Fewest orders      1288951
Name: count, dtype: int64

# 7. Export dataframe as a pickle file to Prepared Data folder.

In [104]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))